# Пример использования поискового движка на русском корпусе Я.Перевода


In [61]:
# Посмотрим на корпус

with open('../../NMT_private_data/1mcorpus/corpus.en_ru.1m.ru') as ru:
    print ('Количество строк в корпусе: %d' % len(ru.readlines()))

Количество строк в корпусе: 1000000


In [56]:
# Первые несколько строк

with open('../../NMT_private_data/1mcorpus/corpus.en_ru.1m.ru') as ru:
    for i in range(5):
        print(ru.readline())
        

Такое развитие характера Гарри может разочаровать читателей, полюбивших его былую мстительность, но с другой стороны это преображение укрепляет позицию тех, кто не видит глубже сюжета и изображения героев.

Решение суда (группа вернулась под крыло к Elektra Entertainment) предотвратило дальнейшие нападки со стороны неугомонного Ульриха и не позволило ему обнародовать детали нового контракта.

Когда тебе 18 или 19 лет, легко перенимать бандитские повадки и переносить их в группу.

А сейчас куча триьютов тем же самым BLACK SABBATH и KISS.

Я был единственным, кто занялся копированием демо на кассете.



# Библиотека Whoosh 

In [69]:
import whoosh

In [68]:
print('Языки, доступные в библиотеке(например для использования встроенных анализаторов):\n' + ', '.join(whoosh.lang.languages))

Языки, доступные в библиотеке(например для использования встроенных анализаторов):
ar, da, nl, en, fi, fr, de, hu, it, no, pt, ro, ru, es, sv, tr


In [75]:
lang_str = 'ru'

In [76]:
print('Реализован ли стеммер в анализаторе для языка: %r' % whoosh.lang.has_stemmer(lang_str))
print('Реализован ли список стоп слов в анализаторе для языка: %r' % whoosh.lang.has_stopwords(lang_str))

Реализован ли стеммер в анализаторе для языка: True
Реализован ли список стоп слов в анализаторе для языка: True


In [79]:
# Зададим анализатор текста для нужного нам языка
from whoosh.analysis import LanguageAnalyzer

analyzer = LanguageAnalyzer(lang_str)

In [80]:
from whoosh.fields import Schema, TEXT, NUMERIC

schema = Schema(id=NUMERIC(stored=True), content=TEXT(analyzer=analyzer, stored=True))

# Построение поискового индекса на основе заданной схемы

Необходимо создать папку для индексов, просто раскомментить код ниже :)


In [93]:
# import os
# os.mkdir("indexdir")

In [94]:
from whoosh.index import create_in

ix = create_in("indexdir", schema)
writer = ix.writer()


Также можно подгружать индекс из уже существующей директории, об этом тут: http://whoosh.readthedocs.io/en/latest/indexing.html

In [95]:
from tqdm import tqdm

with open('../../NMT_private_data/1mcorpus/corpus.en_ru.1m.ru') as ru:
    for i, line in enumerate(tqdm(ru)):
        writer.add_document(id=i, content=line)
writer.commit()     


1000000it [25:38, 649.81it/s]


# Поисковые запросы

In [105]:
# Задание логики парсинга запросов, не нашла подробного описания но что
from whoosh.qparser import QueryParser, OrGroup

# OrGroup makes terms optional by default.
og = OrGroup.factory(0.9)
parser = QueryParser("content", ix.schema, group=og)


BM25 — функция ранжирования, используемая поисковыми системами для упорядочивания документов по их релевантности данному поисковому запросу. Относится к TF-IDF-подобным функциям ранжирования. Аргументы просто из документации

In [107]:
from whoosh.scoring import BM25F

with ix.searcher(weighting=BM25F(B=0.75, content_B=1.0, K1=1.5)) as searcher:
    q = parser.parse('жуки жуют зелёное яблоко')
    results = searcher.search(q, limit=100)
    for i in range(min(len(results), 100)):
        print(str(results[i]['id']) + ': ' + results[i]['content'])
        

243257: Жуки (или это были не жуки?) оказались ничуть не лучше.

392037: Именно здесь меня и убил жук!!!

410775: Сорняки и жуки - враги.

241010: Помимо всего, там было слишком много жуков, а я их ненавижу!!!

396685: А. ЖУКОВ: Уважаемый Дмитрий Анатольевич!

62395: Жук-олень - самое большое насекомое в Великобритании и один из самых известных жуков.

48882: Я ей снесу яблок и шоколаду?

426519: Еще Бернард Шоу сказал: Если у Вас есть яблоко и у меня есть яблоко, и мы обменяемся этими яблоками, то и у Вас и у меня по-прежнему будет по одному яблоку.

894242: Яблоки на дереве, яблоко, солнечный...

340636: - Он коллекционирует жуков и даже написал статьи на эту тему.

319254: Предположим, Фред дает Барни яблоко, после этого у Фреда больше нет яблока.

614960: Первые ароматизированные шары для боулинга с запахом зеленого яблока и цитруса появились весной 2000 года.

696720: Жуков хотел контрнаступление, чтобы продолжить.

752859: Мы против огромного количества жуков.

79205: с минеральн

In [108]:
results.scored_length()

100

Посмотрим на предложении из корпуса

In [109]:
from whoosh.scoring import BM25F

with ix.searcher(weighting=BM25F(B=0.75, content_B=1.0, K1=1.5)) as searcher:
    q = parser.parse('Когда тебе 18 или 19 лет, легко перенимать бандитские повадки и переносить их в группу.')
    results = searcher.search(q, limit=100)
    for i in range(min(len(results), 100)):
        print(str(results[i]['id']) + ': ' + results[i]['content'])
        

2: Когда тебе 18 или 19 лет, легко перенимать бандитские повадки и переносить их в группу.

782928: (61,8%) – молодежь в возрасте 18–19 лет.

881280: Тебе легко говорить.

194786: 19. Не переносите свою личную жизнь на торговлю.

289233: - Тогда сколько тебе лет?

743780: Если нам с тобой будет легко, то и тебе будет легко с нами.

763480: Летом температура поднимается до +35 °C в тени, но из-за низкой влажности жара переносится легко.

597033: 18 Он спросил: "Что же мне дать тебе в залог?"

130484: Уже в 19 лет женился.

63515: Я могу показать тебе более легкий способ".

838930: — Легко тебе здесь будет, парень, или тяжело — все зависит от тебя.

404173: 1. Возраст более чем 18 лет

752836: Он умер, когда ей было 18 лет.

796752: от 18 лет и старше

898555: – Да и вообще, сколько тебе лет?

161113: Особенность этой экспресс-диеты в том, что она довольно легко переносится.

258663: Они построены из дуба и их можно легко переносить с места на место.

474823: 19 раз - тебе нужно было вст